# Projeto 1

Fonte dedados: https://rapidapi.com/mrngstar/api/instagram-scraper-api3

Instagram: isadorabasile (388769829) - https://www.instagram.com/isadorabasile/

## Install dependencies

In [1]:
!pip install requests
!pip install pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 13.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767580 sha256=cbfcfa66f5ec83c4673a8a802c6859152325ceaa57af82d004cf776d826be856
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.
!ls

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,575 kB]
Get:14 https://r2u.stat.illinois.

## Imports dependencies

In [11]:
import requests
import json
import findspark
import os
from google.colab import userdata
import sys

import pandas as pd
import pyspark

from array import ArrayType
from datetime import datetime
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import from_utc_timestamp

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set

In [8]:
url = userdata.get('api_url')

influencer = "isadorabasile"

headers = {
	"x-rapidapi-key": userdata.get('x-rapidapi-key'),
	"x-rapidapi-host": "instagram-scraper-api3.p.rapidapi.com"
}

codigo_ultima_postagem = ''

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [9]:
findspark.init()
findspark.find()

# Create a PySpark session
spark = SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()
spark

## Get infos from Instagram

### Get PROFILE

In [16]:
querystring = {"username_or_id": influencer}

method = 'user_info'

response = requests.get(f'{url}{method}',
                          headers=headers,
                          params=querystring)

response_instagram = response.json()

perfil = response_instagram.get("data")

full_name = perfil.get('full_name')
id = perfil.get('id')
biography = perfil.get('biography')
category = perfil.get('category')
bio_links = perfil.get('bio_links')
follower_count = perfil.get('follower_count')
following_count = perfil.get('following_count')
hd_profile_pic_versions_url = perfil.get('hd_profile_pic_versions')[0].get('url')
media_count = perfil.get('media_count')
is_verified = perfil.get('is_verified')
threads_profile_glyph_url = perfil.get('threads_profile_glyph_url')

schema = T.StructType([
  T.StructField("full_name", T.StringType(), nullable=False),
  T.StructField("id", T.StringType(), nullable=True),
  T.StructField("biography", T.StringType(), nullable=True),
  T.StructField("category", T.StringType(), nullable=True),
  T.StructField("bio_links", T.StringType(), nullable=True),
  T.StructField("follower_count", T.StringType(), nullable=True),
  T.StructField("following_count", T.StringType(), nullable=True),
  T.StructField("hd_profile_pic_versions_url", T.StringType(), nullable=True),
  T.StructField("media_count", T.StringType(), nullable=True),
  T.StructField("is_verified", T.StringType(), nullable=True),
  T.StructField("threads_profile_glyph_url", T.StringType(), nullable=True)
])

print(type(full_name))

data = [(full_name,id,biography,category,bio_links,follower_count,following_count,hd_profile_pic_versions_url,media_count,is_verified,threads_profile_glyph_url)]
print(data)

df = spark.createDataFrame(data, schema=schema)

df = df.withColumn('ts_exec', F.current_timestamp())

# # Mostrando o DataFrame
df.write.partitionBy("full_name","ts_exec").mode("append").format('parquet').save('/content/Datalake/Instagram/Perfil/')

df.show(truncate=False)

<class 'str'>
[('Isadora Basile | Apresentadora de Games 🎮', '388769829', '🎮 Apresentadora |📍 SP\n🎙️ Podcaster | @bomdiaminhavidabr \n🌻 games, esports e gatinhos \n📩 isadorabasile@artentretenimento.com\nLinks 👇', 'Digital creator', [{'link_id': '18061288462650222', 'url': 'http://alura.com.br/isadorabasile', 'lynx_url': 'https://l.instagram.com/?u=http%3A%2F%2Falura.com.br%2Fisadorabasile&e=AT3h3rWwRGOjZMhyvQHFKdNSAZAVRk5BJ6iOuNozhCGHsRQFpOWzfH6rs4IG5FHIJnBvlbtZmRA4Bu6YnjpS99G5_CsPGAXmHe698Wk', 'link_type': 'external', 'title': '', 'image_url': '', 'icon_url': '', 'is_pinned': False, 'is_verified': False, 'open_external_url_with_in_app_browser': True}], 302574, 5671, 'https://instagram.fphl1-1.fna.fbcdn.net/v/t51.2885-19/457242497_1017874039884643_1834676480065307448_n.jpg?stp=dst-jpg_s320x320&_nc_ht=instagram.fphl1-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=b1fPhu5ZaYAQ7kNvgFgnZ5z&_nc_gid=9b7d698bd3e646b2a7e5a6f196decb00&edm=AEF8tYYBAAAA&ccb=7-5&oh=00_AYAMBntoLpbIjEI_ioJfxJa3c4x9AUg2Hkro0eHr2V

### Get POSTS

In [17]:
querystring = {"username_or_id": influencer,"count":"12"}

method = 'user_posts'

response = requests.get(f'{url}{method}',
                          headers=headers,
                          params=querystring)

response_instagram = response.json()

next_max_id = response_instagram.get('data').get('next_max_id')

items = response_instagram.get('data').get('items')

codigo_ultima_postagem = ''

data_posts = []

data = response_instagram.get('data')

for post in items:
  id = post.get('id')
  code = post.get('code')

  # print(f'A postagem é fixada? {post.get("timeline_pinned_user_ids")==None}')
  if post.get('timeline_pinned_user_ids') == None:
    if codigo_ultima_postagem == '':
      codigo_ultima_postagem = code

  device_timestamp = post.get('device_timestamp') # Lembrar de converter
  like_and_view_counts_disabled = post.get('like_and_view_counts_disabled') #?

  usertags = post.get('usertags',{}) #@ marcados

  list_marcados = []

  for marcados in usertags.get('in',[]):
    list_marcados.append({f"username" :marcados.get('user').get('username'),"full_name" :marcados.get('user').get('full_name')})

  caption = post.get('caption')
  image_versions2 = post.get('image_versions2')
  product_type = post.get('product_type')
  coauthor_producers = post.get('coauthor_producers')
  like_count = post.get('like_count')
  comment_count = post.get('comment_count')
  reshare_count = post.get('reshare_count') #encaminhamentos
  timeline_pinned_user_ids = post.get('timeline_pinned_user_ids')

  schema = T.StructType([
    T.StructField("id", T.StringType(), nullable=True),
    T.StructField("code", T.StringType(), nullable=True),
    T.StructField("device_timestamp", T.StringType(), nullable=True),
    T.StructField("like_and_view_counts_disabled", T.StringType(), nullable=True),
    T.StructField("caption", T.StringType(), nullable=True),
    T.StructField("image_versions2", T.StringType(), nullable=True),
    T.StructField("product_type", T.StringType(), nullable=True),
    T.StructField("coauthor_producers", T.StringType(), nullable=True),
    T.StructField("like_count", T.StringType(), nullable=True),
    T.StructField("comment_count", T.StringType(), nullable=True),
    T.StructField("reshare_count", T.StringType(), nullable=True),
    T.StructField("timeline_pinned_user_ids", T.StringType(), nullable=True)
    ])

  data_posts.append((id,code,device_timestamp,like_and_view_counts_disabled,caption,image_versions2,product_type,coauthor_producers,like_count,comment_count,reshare_count,timeline_pinned_user_ids))
  # data_posts.append((id,code))

print(data_posts)

df = spark.createDataFrame(data_posts, schema=schema)

df = df.withColumn('ts_exec', F.current_timestamp())

# # Mostrando o DataFrame
df.write.mode("append").format('parquet').save('/content/Datalake/Instagram/Postagens/')

df.show(truncate=False)

[('3336173774526572597_388769829', 'C5MeF9DoWg1', 1711922696875347, False, {'bit_flags': 0, 'created_at': 1711922929, 'created_at_utc': 1711922929, 'did_report_as_spam': False, 'is_ranked_comment': False, 'pk': '18017790589997457', 'share_enabled': False, 'content_type': 'comment', 'media_id': '3336173774526572597', 'status': 'Active', 'type': 1, 'user_id': '388769829', 'strong_id__': '18017790589997457', 'has_translation': True, 'text': 'NAVI campeã 💛 e mais uma cobertura de major pra conta!!\n\nPrazer IMENSO cobrir o primeiro Major de CS2! Duas semanas dinamarquesas que estarão marcadas no meu coração pra sempre\n\nObrigada @btsbrasiltv pela confiança e por criar uma transmissão com tantos talentos fodas!💚💛 obrigada principalmente aos lindos @renatoestranho e @thitevb que são as estrelas da produção dessa cobertura \U0001faf6\n\nUm beijo e até o próximo Major 💛', 'user': {'pk': '388769829', 'pk_id': '388769829', 'id': '388769829', 'full_name': 'Isadora Basile | Apresentadora de Games

In [13]:
spark.read.parquet('/content/Datalake/Instagram/Postagens/').createOrReplaceTempView('postagens')

spark.sql('select caption,like_count,comment_count  from postagens').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Get COMMENTS

In [14]:
querystring = {"code_or_id_or_url":codigo_ultima_postagem,"sort_order":"popular","min_id":None}

method = 'media_comments'
min_id = None

aux = 1

data_comments = []

while True:
  print('Executando')
  print(aux)
  aux = aux+1
  querystring = {"code_or_id_or_url":codigo_ultima_postagem,"sort_order":"recent","min_id":min_id}
  print(f'parameters: {querystring}')

  response = requests.get(f'{url}{method}',
                        headers=headers,
                        params=querystring)


  response_instagram = response.json()

  array_comments = response_instagram.get('data').get('comments')

  for comment in array_comments:
    pk = comment.get('pk')
    user_id = comment.get('user_id')
    text = comment.get('text')
    comment_like_count = comment.get('comment_like_count',0)
    child_comment_count = comment.get('child_comment_count',0)
    user_username = comment.get('user').get('username')
    user_full_name = comment.get('user').get('full_name')
    user_is_verified = comment.get('user').get('is_verified')
    data_comments.append((pk,user_id,text,comment_like_count,child_comment_count,user_username,user_full_name,user_is_verified))

  print(f'min_id_antes: {min_id}')
  min_id = response_instagram.get('data').get('next_min_id')
  print(f'min_id_depois: {min_id}')


  # Se não houver mais itens a serem buscados, encerra o loop
  if not min_id:
    break

schema = T.StructType([
  T.StructField("pk", T.StringType(), nullable=True),
  T.StructField("user_id", T.StringType(), nullable=True),
  T.StructField("text", T.StringType(), nullable=True),
  T.StructField("comment_like_count", T.StringType(), nullable=True),
  T.StructField("child_comment_count", T.StringType(), nullable=True),
  T.StructField("user_username", T.StringType(), nullable=True),
  T.StructField("user_full_name", T.StringType(), nullable=True),
  T.StructField("user_is_verified", T.StringType(), nullable=True)
])

print(data_comments)

df = spark.createDataFrame(data_comments, schema=schema)

# # Criando o DataFrame
# df = spark.createDataFrame(data, columns)

df = df.withColumn('ts_exec', F.current_timestamp())

# # Mostrando o DataFrame
df.write.mode("append").format('parquet').save('/content/Datalake/Instagram/comentarios/')

df.show(truncate=False)

Executando
1
parameters: {'code_or_id_or_url': 'C_6HkWsygg-', 'sort_order': 'recent', 'min_id': None}
min_id_antes: None
min_id_depois: {"server_cursor": "QVFCc0U4YzgzcjQwSzh2YkVQYWRmaFozc0QzMXZKWlE4QjFfSF9TMXBieGFwekpvaklyUG1JWi1Ualc1aC1OTUtLMnVTZVJRTUFueU5nZDhETC1vejNmUjJxZGFBOGRYM2dfTF9DNkhGUm9RSmc=", "is_server_cursor_inverse": true}
Executando
2
parameters: {'code_or_id_or_url': 'C_6HkWsygg-', 'sort_order': 'recent', 'min_id': '{"server_cursor": "QVFCc0U4YzgzcjQwSzh2YkVQYWRmaFozc0QzMXZKWlE4QjFfSF9TMXBieGFwekpvaklyUG1JWi1Ualc1aC1OTUtLMnVTZVJRTUFueU5nZDhETC1vejNmUjJxZGFBOGRYM2dfTF9DNkhGUm9RSmc=", "is_server_cursor_inverse": true}'}
min_id_antes: {"server_cursor": "QVFCc0U4YzgzcjQwSzh2YkVQYWRmaFozc0QzMXZKWlE4QjFfSF9TMXBieGFwekpvaklyUG1JWi1Ualc1aC1OTUtLMnVTZVJRTUFueU5nZDhETC1vejNmUjJxZGFBOGRYM2dfTF9DNkhGUm9RSmc=", "is_server_cursor_inverse": true}
min_id_depois: {"server_cursor": "QVFBdHZGbUktYkVFWUJYU1YwQXREbjBQQTI2X0lxOTlsLWZTQXFnMERTWmEwWHN4VUE5c0thZDVQVjBOeXptUnQ0R3Z3dlRDejB5Qm92U

In [15]:
spark.read.parquet('/content/Datalake/Instagram/comentarios/').createOrReplaceTempView('comentarios')

spark.sql('select text from comentarios').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                    |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Ai Dodora.. Com essa voz vc faz mais q muita banda nesse Rock in Rio🤤🥹                                                                                                                                

## Analytics

In [18]:
spark.read.parquet('/content/Datalake/Instagram/Perfil/')\
  .select('full_name','follower_count','ts_exec')\
  .where("id!='3444000070552872069_482001976'")\
  .orderBy('ts_exec',ascending=True)\
  .show(truncate=False)

+------------------------------------------+--------------+----------------------+
|full_name                                 |follower_count|ts_exec               |
+------------------------------------------+--------------+----------------------+
|Isadora Basile | Apresentadora de Games 🎮|302574        |2024-09-16 00:38:13.13|
+------------------------------------------+--------------+----------------------+



##Salvar o DataFrame e exportar para o Google Sheets

In [23]:
# Função para converter DataFrame do PySpark para Pandas
def convert_to_pandas(df_spark):
    # Identificar as colunas de timestamp e convertê-las para string
    for col_name, dtype in df_spark.dtypes:
        if dtype == 'timestamp':
            df_spark = df_spark.withColumn(col_name, F.col(col_name).cast('string'))

    # Agora converte o DataFrame do PySpark para Pandas
    df_pandas = df_spark.toPandas()
    return df_pandas

# It looks like you were trying to read in data from a parquet file and save it to a variable called 'df_profile_spark'
# This code reads in the data and saves it to that variable.
df_profile_spark = spark.read.parquet('/content/Datalake/Instagram/Perfil/')
df_posts_spark = spark.read.parquet('/content/Datalake/Instagram/Postagens/')
df_comments_spark = spark.read.parquet('/content/Datalake/Instagram/comentarios/')

# Aplicando a função para os três DataFrames
df_profile_pandas = convert_to_pandas(df_profile_spark)
df_comments_pandas = convert_to_pandas(df_comments_spark)
df_posts_pandas = convert_to_pandas(df_posts_spark)

In [24]:
df_profile_pandas.head()

,id,biography,category,bio_links,follower_count,following_count,hd_profile_pic_versions_url,media_count,is_verified,threads_profile_glyph_url,full_name,ts_exec
0,388769829,🎮 Apresentadora |📍 SP\n🎙️ Podcaster | @bomdiam...,Digital creator,"[{icon_url=, link_type=external, open_external...",302574,5671,https://instagram.fphl1-1.fna.fbcdn.net/v/t51....,325,true,https://www.threads.net/@isadorabasile?modal=t...,Isadora Basile | Apresentadora de Games 🎮,2024-09-16 00:38:13.13


In [25]:
df_posts_pandas.head()

,id,code,device_timestamp,like_and_view_counts_disabled,caption,image_versions2,product_type,coauthor_producers,like_count,comment_count,reshare_count,timeline_pinned_user_ids,ts_exec
0,3336173774526572597_388769829,C5MeF9DoWg1,1711922696875347,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],16487,99,None,[388769829],2024-09-16 00:38:30.328
1,3381902552514924150_388769829,C7u7nQTo452,1717374084819974,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],18357,160,None,[388769829],2024-09-16 00:38:30.328
2,3310884984414277857_388769829,C3yoFn8MBDh,1708908097201613,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],26507,214,None,[388769829],2024-09-16 00:38:30.328
3,3457108948560775230_388769829,C_6HkWsygg-,1726339503852079,false,"{private_reply_status=0, share_enabled=false, ...",{scrubber_spritesheet_info_candidates={default...,clips,[],6003,172,203,None,2024-09-16 00:38:30.328
4,3455754309609321694_388769829,C_1Tjw3SfTe,1726178018850005,false,"{private_reply_status=0, share_enabled=false, ...",{scrubber_spritesheet_info_candidates={default...,clips,"[{is_private=false, full_name=Warner Play, pk_...",7198,174,426,None,2024-09-16 00:38:30.328


In [26]:
df_comments_pandas.head()

,pk,user_id,text,comment_like_count,child_comment_count,user_username,user_full_name,user_is_verified,ts_exec
0,17873293548171749,4184209397,Ai Dodora.. Com essa voz vc faz mais q muita b...,0,0,rodrigo_braga88,Rodrigo Queiroz,false,2024-09-16 00:36:22.569
1,18008457014636165,2139341486,Deve ser por isso que desde então todo COD é u...,0,0,therodcr,Rodrigo Cr,false,2024-09-16 00:36:22.569
2,18017232932578130,292183132,"Linda, inteligente, voz sensual e fofoqueira k...",0,0,brandonsferreira,Brandon Ferreira,false,2024-09-16 00:36:22.569
3,18007887347410304,8615890413,Cara a voz dessa mina é top demais,0,0,jacielgoncalo,Jaciel Gonçalo,false,2024-09-16 00:36:22.569
4,17905111688947399,47900860101,Narrativa perfeita!👏😍😍,0,0,alexandre_c_silva_,Alexandre Candido,false,2024-09-16 00:36:22.569


In [27]:
# Salvar os DataFrames em CSV
df_profile_pandas.to_csv('/content/métricas_influenciador_perfil.csv', index=False)
df_comments_pandas.to_csv('/content/métricas_influenciador_comentarios.csv', index=False)
df_posts_pandas.to_csv('/content/métricas_influenciador_postagens.csv', index=False)

##Calcular as métricas essenciais (engajamento, crescimento de seguidores e análise de conteúdo):

In [28]:
# Calcular engajamento para cada postagem
df_posts_pandas['engagement'] = (df_posts_pandas['like_count'].astype(float) + df_posts_pandas['comment_count'].astype(float)) / df_profile_pandas['follower_count'].astype(float) * 100

# Mostrar o engajamento médio
engagement_avg = df_posts_pandas['engagement'].mean()
print(f"Engajamento médio: {engagement_avg:.2f}%")


Engajamento médio: 5.48%


In [29]:
# Assumindo que você tem o número de seguidores em dois momentos
seguidores_anteriores = 190000  # Exemplo de número anterior de seguidores
seguidores_atuais = df_profile_pandas['follower_count'].astype(int).values[0]

# Calcular o crescimento percentual
crescimento_seguidores = ((seguidores_atuais - seguidores_anteriores) / seguidores_anteriores) * 100
print(f"Crescimento de seguidores: {crescimento_seguidores:.2f}%")

Crescimento de seguidores: 59.25%


In [30]:
# Garantir que as colunas 'like_count' e 'comment_count' sejam numéricas
df_posts_pandas['like_count'] = pd.to_numeric(df_posts_pandas['like_count'], errors='coerce')
df_posts_pandas['comment_count'] = pd.to_numeric(df_posts_pandas['comment_count'], errors='coerce')

# Verifique se há valores nulos e substitua-os por 0 ou outro valor apropriado
df_posts_pandas.fillna({'like_count': 0, 'comment_count': 0}, inplace=True)

# Agora, agrupe por tipo de conteúdo e calcule a média
df_content_analysis = df_posts_pandas.groupby('product_type').agg({
    'like_count': 'mean',
    'comment_count': 'mean'
}).reset_index()

# Renomear as colunas para tornar mais claro
df_content_analysis.columns = ['Tipo de Conteúdo', 'Média de Likes', 'Média de Comentários']

print(df_content_analysis)

     Tipo de Conteúdo  Média de Likes  Média de Comentários
0  carousel_container    20450.333333            157.666667
1               clips     7895.250000            203.625000
2                feed    21347.000000            306.000000


##Salvando os arquivos no drive

In [31]:
# Defina o caminho da pasta no Google Drive onde deseja salvar os arquivos
caminho_pasta = ''

# Certifique-se de que a pasta existe (você pode criar a pasta se necessário)
import os

if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)

# Salvar os DataFrames em arquivos CSV
df_profile_pandas.to_csv(os.path.join(caminho_pasta, 'perfil.csv'), index=False)
df_posts_pandas.to_csv(os.path.join(caminho_pasta, 'postagens.csv'), index=False)
df_comments_pandas.to_csv(os.path.join(caminho_pasta, 'comentarios.csv'), index=False)
df_content_analysis.to_csv(os.path.join(caminho_pasta, 'analise_conteudo.csv'), index=False)